## Getting the required weather information from the hourly text files

### Importing libraries and datasets

In [154]:
import pandas as pd

In [155]:
allstations_merged = pd.read_csv('C:/Users/kiran/OneDrive/Desktop/final/merged_with_target_allstations.csv')

### Creating a new column which contains date,time, weather station id, time which is later be used to search for the required weather information in the hourly text files and extract the same

In [156]:
# Getting only date from 'ActualArrivalTimeStamp' so that we can use this date and the scheduled arrival time
# to get the weather data at that particular time at that particular airport

dates=[]
for i in allstations_merged['ActualArrivalTimeStamp']:
    i=i.split(' ')
    date=i[0]
    time=i[1]
    dates.append(date)

In [169]:
#Getting day information

days=[]
for i in dates:
    j=i.split('-')
    day=j[2]
    days.append(day)

In [174]:
days

['20',
 '10',
 '22',
 '01',
 '15',
 '23',
 '15',
 '16',
 '08',
 '18',
 '11',
 '16',
 '19',
 '17',
 '24',
 '04',
 '04',
 '03',
 '03',
 '17',
 '03',
 '24',
 '30',
 '08',
 '22',
 '05',
 '29',
 '12',
 '18',
 '18',
 '22',
 '19',
 '30',
 '05',
 '13',
 '02',
 '05',
 '05',
 '17',
 '03',
 '12',
 '21',
 '10',
 '18',
 '21',
 '16',
 '13',
 '04',
 '06',
 '14',
 '24',
 '13',
 '21',
 '12',
 '14',
 '03',
 '17',
 '22',
 '02',
 '10',
 '15',
 '08',
 '25',
 '09',
 '08',
 '27',
 '24',
 '30',
 '02',
 '06',
 '27',
 '11',
 '22',
 '08',
 '13',
 '10',
 '27',
 '15',
 '12',
 '31',
 '17',
 '04',
 '21',
 '24',
 '21',
 '05',
 '17',
 '02',
 '17',
 '11',
 '12',
 '13',
 '10',
 '08',
 '17',
 '31',
 '13',
 '27',
 '02',
 '19',
 '08',
 '10',
 '04',
 '07',
 '06',
 '24',
 '02',
 '26',
 '09',
 '30',
 '01',
 '17',
 '09',
 '15',
 '05',
 '27',
 '28',
 '11',
 '27',
 '17',
 '21',
 '08',
 '31',
 '17',
 '06',
 '04',
 '12',
 '10',
 '12',
 '04',
 '29',
 '12',
 '14',
 '12',
 '26',
 '20',
 '01',
 '29',
 '08',
 '27',
 '20',
 '14',
 '18',

In [158]:
allstations_merged.drop('Unnamed: 0', axis=1, inplace=True)

In [159]:
allstations_merged.columns

Index(['FlightNumber', 'Year', 'Month', 'DayOfMonth', 'DayOfWeek',
       'ScheduledDepTime', 'ScheduledArrTime', 'ScheduledTravelTime', 'Origin',
       'Destination', 'Distance', 'ActualArrivalTimeStamp', 'result', 'delay',
       'delay_in_seconds', 'target', 'WeatherStationID_origin',
       'AirportID_origin', 'GroundHeight_origin', 'StationHeight_origin',
       'BarometerHeight_origin', 'Latitude_origin', 'Longitude_origin',
       'TimeZone_origin', 'WeatherStationID_dest', 'AirportID_dest',
       'GroundHeight_dest', 'StationHeight_dest', 'BarometerHeight_dest',
       'Latitude_dest', 'Longitude_dest', 'TimeZone_dest'],
      dtype='object')

### Adding new column in the allstations_merged dataset which contains a string in the format of year-month-day-weatherID-time which is used to perform an exhaustive search on the hourly text files

In [160]:
# using zfill to make sure that we have the same number of places for each data

allstations_merged['WeatherStationID_origin']=allstations_merged['WeatherStationID_origin'].astype(str)
allstations_merged['WeatherStationID_dest']=allstations_merged['WeatherStationID_dest'].astype(str)

allstations_merged['WeatherStationID_origin']=allstations_merged['WeatherStationID_origin'].str.zfill(5)
allstations_merged['WeatherStationID_dest']=allstations_merged['WeatherStationID_dest'].str.zfill(5)

In [161]:
allstations_merged['Month']=allstations_merged['Month'].astype(str)
allstations_merged['DayOfMonth']=allstations_merged['DayOfMonth'].astype(str)
allstations_merged['Month']=allstations_merged['Month'].str.zfill(2)
allstations_merged['DayOfMonth']=allstations_merged['DayOfMonth'].str.zfill(2)

In [162]:
allstations_merged

,FlightNumber,Year,Month,DayOfMonth,DayOfWeek,ScheduledDepTime,ScheduledArrTime,ScheduledTravelTime,Origin,Destination,...,Longitude_origin,TimeZone_origin,WeatherStationID_dest,AirportID_dest,GroundHeight_dest,StationHeight_dest,BarometerHeight_dest,Latitude_dest,Longitude_dest,TimeZone_dest
0,BOS_CLE_29089,2004,09,20,1,1000,1154,114,BOS,CLE,...,-71.01,5,14820,CLE,802,805,805,41.24,-81.51,5
1,BOS_OAK_55049,2004,09,10,5,820,1200,400,BOS,OAK,...,-71.01,5,23230,OAK,85,88,88,37.43,-122.13,8
2,BOS_RSW_47586,2004,03,22,1,700,1036,216,BOS,RSW,...,-71.01,5,12894,RSW,29,30,32,26.32,-81.45,5
3,BOS_PIT_49759,2004,05,01,6,845,1031,106,BOS,PIT,...,-71.01,5,94823,PIT,1172,1203,1175,40.30,-80.14,5
4,BOS_DCA_32791,2004,07,15,4,1300,1430,90,BOS,DCA,...,-71.01,5,13743,DCA,10,65,3,38.52,-77.02,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7856,LAW_DFW_46776,2004,11,24,3,1756,1857,61,LAW,DFW,...,-98.25,6,03927,DFW,559,596,562,32.54,-97.02,6
7857,MTJ_DEN_35253,2004,09,08,3,640,744,64,MTJ,DEN,...,-107.54,7,03017,DEN,5379,5431,5382,39.50,-104.40,7
7858,JNU_SEA_32316,2004,03,18,4,1421,1738,137,JNU,SEA,...,-134.34,9,24233,SEA,447,450,450,47.28,-122.19,8
7859,DRO_DEN_27288,2004,11,17,3,800,914,74,DRO,DEN,...,-107.46,7,03017,DEN,5379,5431,5382,39.50,-104.40,7


In [163]:
allstations_merged['Month']=allstations_merged['Month'].astype(str)
allstations_merged['Year']=allstations_merged['Year'].astype(str)
allstations_merged['WeatherStationID_origin']=allstations_merged['WeatherStationID_origin'].astype(str)
allstations_merged['WeatherStationID_dest']=allstations_merged['WeatherStationID_dest'].astype(str)
allstations_merged['DayOfMonth']=allstations_merged['DayOfMonth'].astype(str)
allstations_merged['ScheduledDepTime']=allstations_merged['ScheduledDepTime'].astype(str)
allstations_merged['ScheduledArrTime']=allstations_merged['ScheduledArrTime'].astype(str)

In [164]:
allstations_merged['ScheduledDepTime']=allstations_merged['ScheduledDepTime'].str.zfill(4)
allstations_merged['ScheduledArrTime']=allstations_merged['ScheduledArrTime'].str.zfill(4)

In [165]:
#creating the required new columns

allstations_merged['origin_neww']=allstations_merged['Year']+allstations_merged['Month']+allstations_merged['DayOfMonth']+allstations_merged['WeatherStationID_origin']+allstations_merged['ScheduledDepTime']

In [172]:
#creating the required new columns

allstations_merged['dest_neww']=allstations_merged['Year']+allstations_merged['Month']+days+allstations_merged['WeatherStationID_dest']+allstations_merged['ScheduledArrTime']

In [173]:
allstations_merged

,FlightNumber,Year,Month,DayOfMonth,DayOfWeek,ScheduledDepTime,ScheduledArrTime,ScheduledTravelTime,Origin,Destination,...,WeatherStationID_dest,AirportID_dest,GroundHeight_dest,StationHeight_dest,BarometerHeight_dest,Latitude_dest,Longitude_dest,TimeZone_dest,origin_neww,dest_neww
0,BOS_CLE_29089,2004,09,20,1,1000,1154,114,BOS,CLE,...,14820,CLE,802,805,805,41.24,-81.51,5,20040920147391000,20040920148201154
1,BOS_OAK_55049,2004,09,10,5,0820,1200,400,BOS,OAK,...,23230,OAK,85,88,88,37.43,-122.13,8,20040910147390820,20040910232301200
2,BOS_RSW_47586,2004,03,22,1,0700,1036,216,BOS,RSW,...,12894,RSW,29,30,32,26.32,-81.45,5,20040322147390700,20040322128941036
3,BOS_PIT_49759,2004,05,01,6,0845,1031,106,BOS,PIT,...,94823,PIT,1172,1203,1175,40.30,-80.14,5,20040501147390845,20040501948231031
4,BOS_DCA_32791,2004,07,15,4,1300,1430,90,BOS,DCA,...,13743,DCA,10,65,3,38.52,-77.02,5,20040715147391300,20040715137431430
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7856,LAW_DFW_46776,2004,11,24,3,1756,1857,61,LAW,DFW,...,03927,DFW,559,596,562,32.54,-97.02,6,20041124039501756,20041124039271857
7857,MTJ_DEN_35253,2004,09,08,3,0640,0744,64,MTJ,DEN,...,03017,DEN,5379,5431,5382,39.50,-104.40,7,20040908930130640,20040908030170744
7858,JNU_SEA_32316,2004,03,18,4,1421,1738,137,JNU,SEA,...,24233,SEA,447,450,450,47.28,-122.19,8,20040318253091421,20040318242331738
7859,DRO_DEN_27288,2004,11,17,3,0800,0914,74,DRO,DEN,...,03017,DEN,5379,5431,5382,39.50,-104.40,7,20041117930050800,20041117030170914


In [175]:
allstations_merged.to_csv('all.csv')

In [176]:
allstations_merged['target'].value_counts()

2    6347
1    1514
Name: target, dtype: int64

In [177]:
import pandas as pd
import numpy as np

### Reading the hourly files as csv

In [178]:
one_hourly=pd.read_csv('C:/Users/kiran/OneDrive/Desktop/hacakthon_final/ProblemStatement_Data&Instructions-1616503373207/DataForPHD_Forall/20210322_weatherdatatoconsider/2004/200401hourly.txt')

In [179]:
two_hourly=pd.read_csv('C:/Users/kiran/OneDrive/Desktop/hacakthon_final/ProblemStatement_Data&Instructions-1616503373207/DataForPHD_Forall/20210322_weatherdatatoconsider/2004/200403hourly.txt')

C:\Users\kiran\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [180]:
three_hourly=pd.read_csv("C:/Users/kiran/OneDrive/Desktop/hacakthon_final/ProblemStatement_Data&Instructions-1616503373207/DataForPHD_Forall/20210322_weatherdatatoconsider/2004/200405hourly.txt")

In [181]:
four_hourly=pd.read_csv('C:/Users/kiran/OneDrive/Desktop/hacakthon_final/ProblemStatement_Data&Instructions-1616503373207/DataForPHD_Forall/20210322_weatherdatatoconsider/2004/200407hourly.txt')

In [182]:
five_hourly=pd.read_csv("C:/Users/kiran/OneDrive/Desktop/hacakthon_final/ProblemStatement_Data&Instructions-1616503373207/DataForPHD_Forall/20210322_weatherdatatoconsider/2004/200409hourly.txt")

In [183]:
six_hourly=pd.read_csv("C:/Users/kiran/OneDrive/Desktop/hacakthon_final/ProblemStatement_Data&Instructions-1616503373207/DataForPHD_Forall/20210322_weatherdatatoconsider/2004/200411hourly.txt")

### Adding new column in all the 6 hourly datasets which contains a string in the format of year-month-day-weatherID-time which is used to match the  data from allstations_merged dataset

In [184]:
one_hourly

,WeatherStationID,YearMonthDay,Time,SkyConditions,Visibility,DBT,DewPointTemp,RelativeHumidityPercent,WindSpeed,WindDirection,WindGustValue,StationPressure
0,3034,20040101,10,OVC095,10SM,30.0,7.0,38.0,0.0,000,0.0,30.10
1,3035,20040101,10,SCT110,10SM,43.0,18.0,37.0,0.0,000,0.0,30.07
2,3039,20040101,10,OVC002,0.25SM,14.0,12.0,92.0,0.0,000,0.0,30.01
3,3040,20040101,10,CLR,10SM,30.0,21.0,69.0,9.0,250,0.0,29.82
4,4128,20040101,10,BKN001 BKN010 OVC024,4SM,NaN,NaN,NaN,0.0,000,0.0,29.80
...,...,...,...,...,...,...,...,...,...,...,...,...
182994,13964,20040104,1153,OVC007,10SM,42.0,40.0,92.0,15.0,260,0.0,29.41
182995,13968,20040104,1153,OVC010,10SM,33.0,30.0,89.0,13.0,340,0.0,29.23
182996,13969,20040104,1153,OVC015,10SM,28.0,22.0,78.0,17.0,320,21.0,28.99
182997,13970,20040104,1153,FEW026,10SM,75.0,64.0,69.0,15.0,150,0.0,29.91


In [185]:
one_hourly['Time']=one_hourly['Time'].astype(str)
one_hourly['Time']=one_hourly['Time'].str.zfill(4)

one_hourly['WeatherStationID']=one_hourly['WeatherStationID'].astype(str)
one_hourly['WeatherStationID']=one_hourly['WeatherStationID'].str.zfill(5)

In [186]:
one_hourly=one_hourly.astype({'WeatherStationID':'str', 'YearMonthDay':'str', 'Time':'str'})

In [187]:
one_hourly['new']=one_hourly['YearMonthDay']+one_hourly['WeatherStationID']+one_hourly['Time']

In [188]:
one_hourly

,WeatherStationID,YearMonthDay,Time,SkyConditions,Visibility,DBT,DewPointTemp,RelativeHumidityPercent,WindSpeed,WindDirection,WindGustValue,StationPressure,new
0,03034,20040101,0010,OVC095,10SM,30.0,7.0,38.0,0.0,000,0.0,30.10,20040101030340010
1,03035,20040101,0010,SCT110,10SM,43.0,18.0,37.0,0.0,000,0.0,30.07,20040101030350010
2,03039,20040101,0010,OVC002,0.25SM,14.0,12.0,92.0,0.0,000,0.0,30.01,20040101030390010
3,03040,20040101,0010,CLR,10SM,30.0,21.0,69.0,9.0,250,0.0,29.82,20040101030400010
4,04128,20040101,0010,BKN001 BKN010 OVC024,4SM,NaN,NaN,NaN,0.0,000,0.0,29.80,20040101041280010
...,...,...,...,...,...,...,...,...,...,...,...,...,...
182994,13964,20040104,1153,OVC007,10SM,42.0,40.0,92.0,15.0,260,0.0,29.41,20040104139641153
182995,13968,20040104,1153,OVC010,10SM,33.0,30.0,89.0,13.0,340,0.0,29.23,20040104139681153
182996,13969,20040104,1153,OVC015,10SM,28.0,22.0,78.0,17.0,320,21.0,28.99,20040104139691153
182997,13970,20040104,1153,FEW026,10SM,75.0,64.0,69.0,15.0,150,0.0,29.91,20040104139701153


In [189]:
two_hourly['Time']=two_hourly['Time'].astype(str)
two_hourly['Time']=two_hourly['Time'].str.zfill(4)

two_hourly['WeatherStationID']=two_hourly['WeatherStationID'].astype(str)
two_hourly['WeatherStationID']=two_hourly['WeatherStationID'].str.zfill(5)


two_hourly=two_hourly.astype({'WeatherStationID':'str', 'YearMonthDay':'str', 'Time':'str'})
two_hourly['new']=two_hourly['YearMonthDay']+two_hourly['WeatherStationID']+two_hourly['Time']

In [190]:
two_hourly

,WeatherStationID,YearMonthDay,Time,SkyConditions,Visibility,DBT,DewPointTemp,RelativeHumidityPercent,WindSpeed,WindDirection,WindGustValue,StationPressure,new
0,03034,20040301,0010,CLR,10SM,30.0,18.0,61.0,8.0,290,0.0,29.93,20040301030340010
1,03035,20040301,0010,CLR,10SM,45.0,23.0,42.0,11.0,250,0.0,29.93,20040301030350010
2,04128,20040301,0010,CLR,8SM,NaN,NaN,NaN,0.0,000,0.0,30.03,20040301041280010
3,04901,20040301,0010,SCT013 BKN019 OVC025,7SM,41.0,37.0,86.0,11.0,120,21.0,29.25,20040301049010010
4,04924,20040301,0010,SCT020 BKN028 OVC075,10SM,45.0,39.0,80.0,16.0,140,23.0,29.40,20040301049240010
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1064788,13729,20040331,0958,BKN005 BKN025 OVC032,2.5SM,NaN,NaN,NaN,0.0,000,0.0,NaN,20040331137290958
1064789,14610,20040331,0958,OVC010,10SM,32.0,27.0,82.0,5.0,VRB,0.0,29.70,20040331146100958
1064790,03702,20040331,0959,SCT007 SCT110,10SM,64.0,61.0,90.0,6.0,100,0.0,29.85,20040331037020959
1064791,03736,20040331,0959,SCT005 OVC039,7SM,NaN,NaN,NaN,4.0,080,0.0,29.93,20040331037360959


In [191]:
three_hourly['Time']=three_hourly['Time'].astype(str)
three_hourly['Time']=three_hourly['Time'].str.zfill(4)

three_hourly['WeatherStationID']=three_hourly['WeatherStationID'].astype(str)
three_hourly['WeatherStationID']=three_hourly['WeatherStationID'].str.zfill(5)

three_hourly=three_hourly.astype({'WeatherStationID':'str', 'YearMonthDay':'str', 'Time':'str'})
three_hourly['new']=three_hourly['YearMonthDay']+three_hourly['WeatherStationID']+three_hourly['Time']

In [192]:
three_hourly

,WeatherStationID,YearMonthDay,Time,SkyConditions,Visibility,DBT,DewPointTemp,RelativeHumidityPercent,WindSpeed,WindDirection,WindGustValue,StationPressure,new
0,03011,20040501,0010,SCT012,10SM,30.0,28.0,92.0,0.0,000,0.0,30.27,20040501030110010
1,03034,20040501,0010,CLR,10SM,37.0,21.0,52.0,7.0,340,0.0,30.20,20040501030340010
2,03035,20040501,0010,BKN090,10SM,54.0,45.0,72.0,8.0,010,0.0,30.18,20040501030350010
3,03950,20040501,0010,BKN016,10SM,NaN,NaN,NaN,13.0,340,21.0,NaN,20040501039500010
4,03998,20040501,0010,OVC010,5SM,NaN,NaN,NaN,10.0,360,0.0,NaN,20040501039980010
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1751551,14898,20040531,0959,BKN010 BKN016 OVC021,2SM,NaN,NaN,NaN,10.0,280,0.0,NaN,20040531148980959
1751552,24154,20040531,0959,BKN001 OVC007,1.5SM,NaN,NaN,NaN,5.0,VRB,0.0,NaN,20040531241540959
1751553,40309,20040531,0959,BKN018CB BKN300,15SM,83.0,77.0,82.0,13.0,150,0.0,29.75,20040531403090959
1751554,93739,20040531,0959,BKN002 OVC060,2SM,NaN,NaN,NaN,8.0,180,0.0,NaN,20040531937390959


In [193]:
four_hourly['Time']=four_hourly['Time'].astype(str)
four_hourly['Time']=four_hourly['Time'].str.zfill(4)

four_hourly['WeatherStationID']=four_hourly['WeatherStationID'].astype(str)
four_hourly['WeatherStationID']=four_hourly['WeatherStationID'].str.zfill(5)

four_hourly=four_hourly.astype({'WeatherStationID':'str', 'YearMonthDay':'str', 'Time':'str'})
four_hourly['new']=four_hourly['YearMonthDay']+four_hourly['WeatherStationID']+four_hourly['Time']

In [194]:
four_hourly

,WeatherStationID,YearMonthDay,Time,SkyConditions,Visibility,DBT,DewPointTemp,RelativeHumidityPercent,WindSpeed,WindDirection,WindGustValue,StationPressure,new
0,03954,20040701,0010,FEW010 BKN037 OVC046,9SM,NaN,NaN,NaN,15,330,0.0,NaN,20040701039540010
1,14748,20040701,0010,VV001,0.25SM,NaN,NaN,NaN,0,000,0.0,NaN,20040701147480010
2,53819,20040701,0010,SCT007,7SM,NaN,NaN,NaN,0,000,0.0,NaN,20040701538190010
3,53872,20040701,0010,SCT120,2SM,NaN,NaN,NaN,0,000,0.0,NaN,20040701538720010
4,53912,20040701,0010,BKN009,9SM,NaN,NaN,NaN,7,150,0.0,NaN,20040701539120010
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1189458,12926,20040731,0959,SCT025 BKN250,7SM,88.0,81.0,80.0,4.0,320,0.0,29.90,20040731129260959
1189459,40309,20040731,0959,OVC016CB,4SM,75.0,73.0,94.0,14.0,280,0.0,29.76,20040731403090959
1189460,53882,20040731,0959,CLR,10SM,90.0,75.0,61.0,4.0,090,0.0,30.01,20040731538820959
1189461,63842,20040731,0959,BKN023 BKN032,10SM,NaN,NaN,NaN,6.0,020,0.0,30.02,20040731638420959


In [195]:
five_hourly['Time']=five_hourly['Time'].astype(str)
five_hourly['Time']=five_hourly['Time'].str.zfill(4)

five_hourly['WeatherStationID']=five_hourly['WeatherStationID'].astype(str)
five_hourly['WeatherStationID']=five_hourly['WeatherStationID'].str.zfill(5)

five_hourly=five_hourly.astype({'WeatherStationID':'str', 'YearMonthDay':'str', 'Time':'str'})
five_hourly['new']=five_hourly['YearMonthDay']+five_hourly['WeatherStationID']+five_hourly['Time']

In [196]:
five_hourly

,WeatherStationID,YearMonthDay,Time,SkyConditions,Visibility,DBT,DewPointTemp,RelativeHumidityPercent,WindSpeed,WindDirection,WindGustValue,StationPressure,new
0,11640,20040901,0010,SCT020 BKN026,10SM,NaN,NaN,NaN,15.0,180,0.0,NaN,20040901116400010
1,13729,20040901,0010,OVC002,2SM,NaN,NaN,NaN,0.0,000,0.0,NaN,20040901137290010
2,53871,20040901,0010,BKN017,2.5SM,NaN,NaN,NaN,0.0,000,0.0,NaN,20040901538710010
3,03820,20040901,1000,OVC012,10SM,NaN,NaN,NaN,3.0,VRB,0.0,NaN,20040901038201000
4,13895,20040901,1000,BKN031 BKN050 OVC080,10SM,NaN,NaN,NaN,4.0,090,0.0,NaN,20040901138951000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1126618,40505,20040930,0959,FEW014 SCT350,15SM,86.0,77.0,76.0,8.0,270,0.0,29.83,20040930405050959
1126619,53817,20040930,0959,CLR,5SM,NaN,NaN,NaN,6.0,230,0.0,30.04,20040930538170959
1126620,63842,20040930,0959,CLR,5SM,NaN,NaN,NaN,0.0,000,0.0,30.03,20040930638420959
1126621,94014,20040930,0959,SCT001,10SM,NaN,NaN,NaN,3.0,040,0.0,NaN,20040930940140959


In [197]:
six_hourly['Time']=six_hourly['Time'].astype(str)
six_hourly['Time']=six_hourly['Time'].str.zfill(4)

six_hourly['WeatherStationID']=six_hourly['WeatherStationID'].astype(str)
six_hourly['WeatherStationID']=six_hourly['WeatherStationID'].str.zfill(5)

six_hourly=six_hourly.astype({'WeatherStationID':'str', 'YearMonthDay':'str', 'Time':'str'})
six_hourly['new']=six_hourly['YearMonthDay']+six_hourly['WeatherStationID']+six_hourly['Time']

In [198]:
six_hourly

,WeatherStationID,YearMonthDay,Time,SkyConditions,Visibility,DBT,DewPointTemp,RelativeHumidityPercent,WindSpeed,WindDirection,WindGustValue,StationPressure,new
0,13838,20041101,0010,SCT023 BKN031 BKN038,10SM,NaN,NaN,NaN,14.0,160,0.0,NaN,20041101138380010
1,26616,20041101,0010,BKN030 OVC042,3SM,NaN,NaN,NaN,18.0,110,0.0,NaN,20041101266160010
2,93919,20041101,0010,CLR,3SM,NaN,NaN,NaN,5.0,150,0.0,NaN,20041101939190010
3,93953,20041101,0010,BKN001 BKN005 OVC024,1SM,NaN,NaN,NaN,13.0,310,16.0,NaN,20041101939530010
4,03013,20041101,1000,BKN014 OVC019,2.5SM,NaN,NaN,NaN,26.0,360,35.0,NaN,20041101030131000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1169738,63815,20041130,0959,SCT010 BKN040 OVC048,7SM,46.0,46.0,100.0,0.0,000,0.0,30.08,20041130638150959
1169739,64752,20041130,0959,CLR,10SM,43.0,32.0,65.0,0.0,000,0.0,30.31,20041130647520959
1169740,94176,20041130,0959,BKN012 OVC020,0.75SM,NaN,NaN,NaN,4.0,240,0.0,NaN,20041130941760959
1169741,94836,20041130,0959,CLR,10SM,25.0,18.0,75.0,10.0,240,0.0,29.97,20041130948360959


### Saving these changes made in the dataframe toa  csv

In [199]:
one_hourly.to_csv('hourly1.csv')

In [200]:
two_hourly.to_csv('hourly2.csv')

In [201]:
three_hourly.to_csv('hourly3.csv')

In [202]:
four_hourly.to_csv('hourly4.csv')

In [203]:
five_hourly.to_csv('hourly5.csv')

In [204]:
six_hourly.to_csv('hourly6.csv')

In [205]:
allstations_merged['origin_neww']

0       20040920147391000
1       20040910147390820
2       20040322147390700
3       20040501147390845
4       20040715147391300
              ...        
7856    20041124039501756
7857    20040908930130640
7858    20040318253091421
7859    20041117930050800
7860    20040518148601335
Name: origin_neww, Length: 7861, dtype: object

In [206]:
allstations_merged['dest_neww']

0       20040920148201154
1       20040910232301200
2       20040322128941036
3       20040501948231031
4       20040715137431430
              ...        
7856    20041124039271857
7857    20040908030170744
7858    20040318242331738
7859    20041117030170914
7860    20040518938141452
Name: dest_neww, Length: 7861, dtype: object

### Writing a logic to perform the exhausive search between the allstations_merged and hourly files

In [207]:
# reading the data from the hourly files

import csv

with open('hourly1.csv', 'r') as file:
    reader1 = csv.reader(file)
    data1 = list(reader1)
    
with open('hourly2.csv', 'r') as file:
    reader2 = csv.reader(file)
    data2 = list(reader2)

with open('hourly3.csv', 'r') as file:
    reader3 = csv.reader(file)
    data3 = list(reader3)
    
with open('hourly4.csv', 'r') as file:
    reader4 = csv.reader(file)
    data4 = list(reader4)
    
with open('hourly5.csv', 'r') as file:
    reader5 = csv.reader(file)
    data5 = list(reader5)
    
with open('hourly6.csv', 'r') as file:
    reader6 = csv.reader(file)
    data6 = list(reader6)

In [208]:
## writing the logic to perform the search operation and extract the required data

origin_data_hourly=[]
count=0
for i in allstations_merged['origin_neww']:
    i=str(i)
    a=i[4:6]
      
    if a =='01':
        data = data1
    elif a =='03':
        data = data2
    elif a == '05':
        data = data3
    elif a =='07':
        data = data4
    elif a == '09':
        data = data5
    else :
        data = data6
        
    target_value = int(i)
    column_index = 13
    closest_value = None
        
    for row in data[1:]:           
        value = int(row[column_index])
        if closest_value is None or abs(value - target_value) < abs(closest_value - target_value):
            closest_value = value
            row_info = row

    count=count+1
    print(count)
    origin_data_hourly.append(row_info)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060


3500
3501
3502
3503
3504
3505
3506
3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699


5139
5140
5141
5142
5143
5144
5145
5146
5147
5148
5149
5150
5151
5152
5153
5154
5155
5156
5157
5158
5159
5160
5161
5162
5163
5164
5165
5166
5167
5168
5169
5170
5171
5172
5173
5174
5175
5176
5177
5178
5179
5180
5181
5182
5183
5184
5185
5186
5187
5188
5189
5190
5191
5192
5193
5194
5195
5196
5197
5198
5199
5200
5201
5202
5203
5204
5205
5206
5207
5208
5209
5210
5211
5212
5213
5214
5215
5216
5217
5218
5219
5220
5221
5222
5223
5224
5225
5226
5227
5228
5229
5230
5231
5232
5233
5234
5235
5236
5237
5238
5239
5240
5241
5242
5243
5244
5245
5246
5247
5248
5249
5250
5251
5252
5253
5254
5255
5256
5257
5258
5259
5260
5261
5262
5263
5264
5265
5266
5267
5268
5269
5270
5271
5272
5273
5274
5275
5276
5277
5278
5279
5280
5281
5282
5283
5284
5285
5286
5287
5288
5289
5290
5291
5292
5293
5294
5295
5296
5297
5298
5299
5300
5301
5302
5303
5304
5305
5306
5307
5308
5309
5310
5311
5312
5313
5314
5315
5316
5317
5318
5319
5320
5321
5322
5323
5324
5325
5326
5327
5328
5329
5330
5331
5332
5333
5334
5335
5336
5337
5338


6778
6779
6780
6781
6782
6783
6784
6785
6786
6787
6788
6789
6790
6791
6792
6793
6794
6795
6796
6797
6798
6799
6800
6801
6802
6803
6804
6805
6806
6807
6808
6809
6810
6811
6812
6813
6814
6815
6816
6817
6818
6819
6820
6821
6822
6823
6824
6825
6826
6827
6828
6829
6830
6831
6832
6833
6834
6835
6836
6837
6838
6839
6840
6841
6842
6843
6844
6845
6846
6847
6848
6849
6850
6851
6852
6853
6854
6855
6856
6857
6858
6859
6860
6861
6862
6863
6864
6865
6866
6867
6868
6869
6870
6871
6872
6873
6874
6875
6876
6877
6878
6879
6880
6881
6882
6883
6884
6885
6886
6887
6888
6889
6890
6891
6892
6893
6894
6895
6896
6897
6898
6899
6900
6901
6902
6903
6904
6905
6906
6907
6908
6909
6910
6911
6912
6913
6914
6915
6916
6917
6918
6919
6920
6921
6922
6923
6924
6925
6926
6927
6928
6929
6930
6931
6932
6933
6934
6935
6936
6937
6938
6939
6940
6941
6942
6943
6944
6945
6946
6947
6948
6949
6950
6951
6952
6953
6954
6955
6956
6957
6958
6959
6960
6961
6962
6963
6964
6965
6966
6967
6968
6969
6970
6971
6972
6973
6974
6975
6976
6977


In [209]:
origin_data_hourly

[['749904',
  '14739',
  '20040920',
  '0954',
  'FEW250',
  '10SM',
  '61.0',
  '41.0',
  '48.0',
  '8',
  '030',
  '0.0',
  '30.31',
  '20040920147390954'],
 ['374124',
  '14739',
  '20040910',
  '0854',
  'SCT034',
  '10SM',
  '70.0',
  '55.0',
  '59.0',
  '18.0',
  '280',
  '22.0',
  '29.92',
  '20040910147390854'],
 ['745982',
  '14739',
  '20040322',
  '0654',
  'CLR',
  '10SM',
  '24.0',
  '7.0',
  '48.0',
  '18.0',
  '300',
  '24.0',
  '29.91',
  '20040322147390654'],
 ['56753',
  '14739',
  '20040501',
  '0854',
  'SCT300',
  '10SM',
  '70.0',
  '53.0',
  '55.0',
  '10.0',
  '210',
  '0.0',
  '30.1',
  '20040501147390854'],
 ['538278',
  '14739',
  '20040715',
  '1254',
  'FEW028 SCT037 BKN048',
  '8SM',
  '67.0',
  '62.0',
  '84.0',
  '10.0',
  '080',
  '0.0',
  '29.54',
  '20040715147391254'],
 ['890180',
  '14739',
  '20041123',
  '0854',
  'CLR',
  '10SM',
  '43.0',
  '34.0',
  '71.0',
  '3.0',
  '040',
  '0.0',
  '30.15',
  '20041123147390854'],
 ['559697',
  '14739',
  '

In [210]:
six_hourly.columns

Index(['WeatherStationID', 'YearMonthDay', 'Time', 'SkyConditions',
       'Visibility', 'DBT', 'DewPointTemp', 'RelativeHumidityPercent',
       'WindSpeed', 'WindDirection', 'WindGustValue', 'StationPressure',
       'new'],
      dtype='object')

### Merging the found rows and saving it in a csv

In [211]:
origin_hourly_data = pd.DataFrame(origin_data_hourly, columns=['a','WeatherStationID_o', 'YearMonthDay_o', 'Time_o', 'SkyConditions_o',
       'Visibility_o', 'DBT_o', 'DewPointTemp_o', 'RelativeHumidityPercent_o',
       'WindSpeed_o', 'WindDirection_o', 'WindGustValue_o', 'StationPressure_o',
       'new_o'])

In [212]:
origin_hourly_data

,a,WeatherStationID_o,YearMonthDay_o,Time_o,SkyConditions_o,Visibility_o,DBT_o,DewPointTemp_o,RelativeHumidityPercent_o,WindSpeed_o,WindDirection_o,WindGustValue_o,StationPressure_o,new_o
0,749904,14739,20040920,0954,FEW250,10SM,61.0,41.0,48.0,8,030,0.0,30.31,20040920147390954
1,374124,14739,20040910,0854,SCT034,10SM,70.0,55.0,59.0,18.0,280,22.0,29.92,20040910147390854
2,745982,14739,20040322,0654,CLR,10SM,24.0,7.0,48.0,18.0,300,24.0,29.91,20040322147390654
3,56753,14739,20040501,0854,SCT300,10SM,70.0,53.0,55.0,10.0,210,0.0,30.1,20040501147390854
4,538278,14739,20040715,1254,FEW028 SCT037 BKN048,8SM,67.0,62.0,84.0,10.0,080,0.0,29.54,20040715147391254
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7856,907888,03950,20041124,1753,CLR,10SM,44.0,24.0,45.0,4.0,320,0.0,28.89,20041124039501753
7857,295823,93013,20040908,0653,CLR,10SM,52.0,39.0,61.0,5.0,190,0.0,24.51,20040908930130653
7858,584107,25309,20040318,1453,FEW050 BKN200,10SM,39.0,11.0,31.0,4.0,100,0.0,29.29,20040318253091453
7859,645289,93005,20041117,0801,SCT001 OVC006,0.25SM,,,,0.0,000,0.0,,20041117930050801


In [213]:
origin_hourly_data=origin_hourly_data.drop('a', axis=1)

In [214]:
origin_hourly_data.columns

Index(['WeatherStationID_o', 'YearMonthDay_o', 'Time_o', 'SkyConditions_o',
       'Visibility_o', 'DBT_o', 'DewPointTemp_o', 'RelativeHumidityPercent_o',
       'WindSpeed_o', 'WindDirection_o', 'WindGustValue_o',
       'StationPressure_o', 'new_o'],
      dtype='object')

In [215]:
origin_hourly_data

,WeatherStationID_o,YearMonthDay_o,Time_o,SkyConditions_o,Visibility_o,DBT_o,DewPointTemp_o,RelativeHumidityPercent_o,WindSpeed_o,WindDirection_o,WindGustValue_o,StationPressure_o,new_o
0,14739,20040920,0954,FEW250,10SM,61.0,41.0,48.0,8,030,0.0,30.31,20040920147390954
1,14739,20040910,0854,SCT034,10SM,70.0,55.0,59.0,18.0,280,22.0,29.92,20040910147390854
2,14739,20040322,0654,CLR,10SM,24.0,7.0,48.0,18.0,300,24.0,29.91,20040322147390654
3,14739,20040501,0854,SCT300,10SM,70.0,53.0,55.0,10.0,210,0.0,30.1,20040501147390854
4,14739,20040715,1254,FEW028 SCT037 BKN048,8SM,67.0,62.0,84.0,10.0,080,0.0,29.54,20040715147391254
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7856,03950,20041124,1753,CLR,10SM,44.0,24.0,45.0,4.0,320,0.0,28.89,20041124039501753
7857,93013,20040908,0653,CLR,10SM,52.0,39.0,61.0,5.0,190,0.0,24.51,20040908930130653
7858,25309,20040318,1453,FEW050 BKN200,10SM,39.0,11.0,31.0,4.0,100,0.0,29.29,20040318253091453
7859,93005,20041117,0801,SCT001 OVC006,0.25SM,,,,0.0,000,0.0,,20041117930050801


In [216]:
origin_hourly_data.to_csv('hourly_origin_data.csv')

In [217]:
dest_data_hourly=[]
count=0
for i in allstations_merged['dest_neww']:
    i=str(i)
    a=i[4:6]
      
    if a =='01':
        data = data1
    elif a =='03':
        data = data2
    elif a == '05':
        data = data3
    elif a =='07':
        data = data4
    elif a == '09':
        data = data5
    else :
        data = data6
        
    target_value = int(i)
    column_index = 13
    closest_value = None
        
    for row in data[1:]:           
        value = int(row[column_index])
        if closest_value is None or abs(value - target_value) < abs(closest_value - target_value):
            closest_value = value
            row_info = row

    count=count+1
    print(count)
    dest_data_hourly.append(row_info)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060


3500
3501
3502
3503
3504
3505
3506
3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699


5139
5140
5141
5142
5143
5144
5145
5146
5147
5148
5149
5150
5151
5152
5153
5154
5155
5156
5157
5158
5159
5160
5161
5162
5163
5164
5165
5166
5167
5168
5169
5170
5171
5172
5173
5174
5175
5176
5177
5178
5179
5180
5181
5182
5183
5184
5185
5186
5187
5188
5189
5190
5191
5192
5193
5194
5195
5196
5197
5198
5199
5200
5201
5202
5203
5204
5205
5206
5207
5208
5209
5210
5211
5212
5213
5214
5215
5216
5217
5218
5219
5220
5221
5222
5223
5224
5225
5226
5227
5228
5229
5230
5231
5232
5233
5234
5235
5236
5237
5238
5239
5240
5241
5242
5243
5244
5245
5246
5247
5248
5249
5250
5251
5252
5253
5254
5255
5256
5257
5258
5259
5260
5261
5262
5263
5264
5265
5266
5267
5268
5269
5270
5271
5272
5273
5274
5275
5276
5277
5278
5279
5280
5281
5282
5283
5284
5285
5286
5287
5288
5289
5290
5291
5292
5293
5294
5295
5296
5297
5298
5299
5300
5301
5302
5303
5304
5305
5306
5307
5308
5309
5310
5311
5312
5313
5314
5315
5316
5317
5318
5319
5320
5321
5322
5323
5324
5325
5326
5327
5328
5329
5330
5331
5332
5333
5334
5335
5336
5337
5338


6778
6779
6780
6781
6782
6783
6784
6785
6786
6787
6788
6789
6790
6791
6792
6793
6794
6795
6796
6797
6798
6799
6800
6801
6802
6803
6804
6805
6806
6807
6808
6809
6810
6811
6812
6813
6814
6815
6816
6817
6818
6819
6820
6821
6822
6823
6824
6825
6826
6827
6828
6829
6830
6831
6832
6833
6834
6835
6836
6837
6838
6839
6840
6841
6842
6843
6844
6845
6846
6847
6848
6849
6850
6851
6852
6853
6854
6855
6856
6857
6858
6859
6860
6861
6862
6863
6864
6865
6866
6867
6868
6869
6870
6871
6872
6873
6874
6875
6876
6877
6878
6879
6880
6881
6882
6883
6884
6885
6886
6887
6888
6889
6890
6891
6892
6893
6894
6895
6896
6897
6898
6899
6900
6901
6902
6903
6904
6905
6906
6907
6908
6909
6910
6911
6912
6913
6914
6915
6916
6917
6918
6919
6920
6921
6922
6923
6924
6925
6926
6927
6928
6929
6930
6931
6932
6933
6934
6935
6936
6937
6938
6939
6940
6941
6942
6943
6944
6945
6946
6947
6948
6949
6950
6951
6952
6953
6954
6955
6956
6957
6958
6959
6960
6961
6962
6963
6964
6965
6966
6967
6968
6969
6970
6971
6972
6973
6974
6975
6976
6977


In [220]:
allstations_merged['dest_neww']

0       20040920148201154
1       20040910232301200
2       20040322128941036
3       20040501948231031
4       20040715137431430
              ...        
7856    20041124039271857
7857    20040908030170744
7858    20040318242331738
7859    20041117030170914
7860    20040518938141452
Name: dest_neww, Length: 7861, dtype: object

In [218]:
dest_data_hourly

[['716017',
  '14820',
  '20040920',
  '1151',
  'FEW250',
  '10SM',
  '71.0',
  '35.0',
  '27.0',
  '8.0',
  '190',
  '0.0',
  '29.49',
  '20040920148201151'],
 ['341987',
  '23230',
  '20040910',
  '1153',
  'CLR',
  '10SM',
  '69.0',
  '54.0',
  '59.0',
  '11.0',
  '290',
  '0.0',
  '29.86',
  '20040910232301153'],
 ['717515',
  '12894',
  '20040322',
  '1030',
  'FEW019',
  '10SM',
  '',
  '',
  '',
  '6.0',
  '050',
  '0.0',
  '',
  '20040322128941030'],
 ['1478',
  '94823',
  '20040501',
  '1051',
  'SCT031 BKN080',
  '10SM',
  '72.0',
  '59.0',
  '64.0',
  '9.0',
  '220',
  '0.0',
  '28.76',
  '20040501948231051'],
 ['540838',
  '13743',
  '20040715',
  '1451',
  'SCT060 BKN095',
  '10SM',
  '83.0',
  '56.0',
  '40.0',
  '13.0',
  '260',
  '0.0',
  '29.69',
  '20040715137431451'],
 ['850775',
  '13881',
  '20041123',
  '1151',
  'SCT006 BKN009 OVC016',
  '3SM',
  '64.0',
  '61.0',
  '90.0',
  '4.0',
  'VRB',
  '0.0',
  '29.19',
  '20041123138811151'],
 ['526016',
  '23129',
  '2

In [221]:
six_hourly.columns

Index(['WeatherStationID', 'YearMonthDay', 'Time', 'SkyConditions',
       'Visibility', 'DBT', 'DewPointTemp', 'RelativeHumidityPercent',
       'WindSpeed', 'WindDirection', 'WindGustValue', 'StationPressure',
       'new'],
      dtype='object')

In [222]:
dest_hourly_data = pd.DataFrame(dest_data_hourly, columns=['a','WeatherStationID_d', 'YearMonthDay_d', 'Time_d', 'SkyConditions_d',
       'Visibility_d', 'DBT_d', 'DewPointTemp_d', 'RelativeHumidityPercent_d',
       'WindSpeed_d', 'WindDirection_d', 'WindGustValue_d', 'StationPressure_d',
       'new_d'])

In [223]:
dest_hourly_data

,a,WeatherStationID_d,YearMonthDay_d,Time_d,SkyConditions_d,Visibility_d,DBT_d,DewPointTemp_d,RelativeHumidityPercent_d,WindSpeed_d,WindDirection_d,WindGustValue_d,StationPressure_d,new_d
0,716017,14820,20040920,1151,FEW250,10SM,71.0,35.0,27.0,8.0,190,0.0,29.49,20040920148201151
1,341987,23230,20040910,1153,CLR,10SM,69.0,54.0,59.0,11.0,290,0.0,29.86,20040910232301153
2,717515,12894,20040322,1030,FEW019,10SM,,,,6.0,050,0.0,,20040322128941030
3,1478,94823,20040501,1051,SCT031 BKN080,10SM,72.0,59.0,64.0,9.0,220,0.0,28.76,20040501948231051
4,540838,13743,20040715,1451,SCT060 BKN095,10SM,83.0,56.0,40.0,13.0,260,0.0,29.69,20040715137431451
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7856,909681,03927,20041124,1853,CLR,10SM,49.0,33.0,55.0,9.0,290,0.0,29.45,20041124039271853
7857,297059,03017,20040908,0753,FEW080 SCT130,10SM,68.0,42.0,39.0,10.0,240,0.0,24.77,20040908030170753
7858,589718,24233,20040318,1756,FEW040 SCT050 BKN100,10SM,43.0,30.0,60.0,17.0,230,24.0,29.34,20040318242331756
7859,647670,03017,20041117,0953,FEW080 SCT120 BKN250,10SM,51.0,30.0,45.0,6.0,250,0.0,24.79,20041117030170953


In [224]:
dest_hourly_data=dest_hourly_data.drop('a', axis=1)

In [225]:
dest_hourly_data.columns

Index(['WeatherStationID_d', 'YearMonthDay_d', 'Time_d', 'SkyConditions_d',
       'Visibility_d', 'DBT_d', 'DewPointTemp_d', 'RelativeHumidityPercent_d',
       'WindSpeed_d', 'WindDirection_d', 'WindGustValue_d',
       'StationPressure_d', 'new_d'],
      dtype='object')

In [226]:
dest_hourly_data

,WeatherStationID_d,YearMonthDay_d,Time_d,SkyConditions_d,Visibility_d,DBT_d,DewPointTemp_d,RelativeHumidityPercent_d,WindSpeed_d,WindDirection_d,WindGustValue_d,StationPressure_d,new_d
0,14820,20040920,1151,FEW250,10SM,71.0,35.0,27.0,8.0,190,0.0,29.49,20040920148201151
1,23230,20040910,1153,CLR,10SM,69.0,54.0,59.0,11.0,290,0.0,29.86,20040910232301153
2,12894,20040322,1030,FEW019,10SM,,,,6.0,050,0.0,,20040322128941030
3,94823,20040501,1051,SCT031 BKN080,10SM,72.0,59.0,64.0,9.0,220,0.0,28.76,20040501948231051
4,13743,20040715,1451,SCT060 BKN095,10SM,83.0,56.0,40.0,13.0,260,0.0,29.69,20040715137431451
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7856,03927,20041124,1853,CLR,10SM,49.0,33.0,55.0,9.0,290,0.0,29.45,20041124039271853
7857,03017,20040908,0753,FEW080 SCT130,10SM,68.0,42.0,39.0,10.0,240,0.0,24.77,20040908030170753
7858,24233,20040318,1756,FEW040 SCT050 BKN100,10SM,43.0,30.0,60.0,17.0,230,24.0,29.34,20040318242331756
7859,03017,20041117,0953,FEW080 SCT120 BKN250,10SM,51.0,30.0,45.0,6.0,250,0.0,24.79,20041117030170953


In [227]:
dest_hourly_data.to_csv('hourly_dest_data.csv')

In [228]:
df=pd.read_csv('merged_with_target_allstations.csv')

In [229]:
df=df.drop('Unnamed: 0', axis=1)

In [230]:
df

,FlightNumber,Year,Month,DayOfMonth,DayOfWeek,ScheduledDepTime,ScheduledArrTime,ScheduledTravelTime,Origin,Destination,...,Longitude_origin,TimeZone_origin,WeatherStationID_dest,AirportID_dest,GroundHeight_dest,StationHeight_dest,BarometerHeight_dest,Latitude_dest,Longitude_dest,TimeZone_dest
0,BOS_CLE_29089,2004,9,20,1,1000,1154,114,BOS,CLE,...,-71.01,5,14820,CLE,802,805,805,41.24,-81.51,5
1,BOS_OAK_55049,2004,9,10,5,820,1200,400,BOS,OAK,...,-71.01,5,23230,OAK,85,88,88,37.43,-122.13,8
2,BOS_RSW_47586,2004,3,22,1,700,1036,216,BOS,RSW,...,-71.01,5,12894,RSW,29,30,32,26.32,-81.45,5
3,BOS_PIT_49759,2004,5,1,6,845,1031,106,BOS,PIT,...,-71.01,5,94823,PIT,1172,1203,1175,40.30,-80.14,5
4,BOS_DCA_32791,2004,7,15,4,1300,1430,90,BOS,DCA,...,-71.01,5,13743,DCA,10,65,3,38.52,-77.02,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7856,LAW_DFW_46776,2004,11,24,3,1756,1857,61,LAW,DFW,...,-98.25,6,3927,DFW,559,596,562,32.54,-97.02,6
7857,MTJ_DEN_35253,2004,9,8,3,640,744,64,MTJ,DEN,...,-107.54,7,3017,DEN,5379,5431,5382,39.50,-104.40,7
7858,JNU_SEA_32316,2004,3,18,4,1421,1738,137,JNU,SEA,...,-134.34,9,24233,SEA,447,450,450,47.28,-122.19,8
7859,DRO_DEN_27288,2004,11,17,3,800,914,74,DRO,DEN,...,-107.46,7,3017,DEN,5379,5431,5382,39.50,-104.40,7


In [231]:
df['WeatherStationID_origin']=df['WeatherStationID_origin'].astype(str)

In [232]:
origin_hourly_data.columns

Index(['WeatherStationID_o', 'YearMonthDay_o', 'Time_o', 'SkyConditions_o',
       'Visibility_o', 'DBT_o', 'DewPointTemp_o', 'RelativeHumidityPercent_o',
       'WindSpeed_o', 'WindDirection_o', 'WindGustValue_o',
       'StationPressure_o', 'new_o'],
      dtype='object')

In [233]:
origin_df=pd.concat([df,origin_hourly_data], axis=1, join='outer')

In [234]:
df['WeatherStationID_dest']=df['WeatherStationID_dest'].astype(str)

In [235]:
origin_df

,FlightNumber,Year,Month,DayOfMonth,DayOfWeek,ScheduledDepTime,ScheduledArrTime,ScheduledTravelTime,Origin,Destination,...,SkyConditions_o,Visibility_o,DBT_o,DewPointTemp_o,RelativeHumidityPercent_o,WindSpeed_o,WindDirection_o,WindGustValue_o,StationPressure_o,new_o
0,BOS_CLE_29089,2004,9,20,1,1000,1154,114,BOS,CLE,...,FEW250,10SM,61.0,41.0,48.0,8,030,0.0,30.31,20040920147390954
1,BOS_OAK_55049,2004,9,10,5,820,1200,400,BOS,OAK,...,SCT034,10SM,70.0,55.0,59.0,18.0,280,22.0,29.92,20040910147390854
2,BOS_RSW_47586,2004,3,22,1,700,1036,216,BOS,RSW,...,CLR,10SM,24.0,7.0,48.0,18.0,300,24.0,29.91,20040322147390654
3,BOS_PIT_49759,2004,5,1,6,845,1031,106,BOS,PIT,...,SCT300,10SM,70.0,53.0,55.0,10.0,210,0.0,30.1,20040501147390854
4,BOS_DCA_32791,2004,7,15,4,1300,1430,90,BOS,DCA,...,FEW028 SCT037 BKN048,8SM,67.0,62.0,84.0,10.0,080,0.0,29.54,20040715147391254
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7856,LAW_DFW_46776,2004,11,24,3,1756,1857,61,LAW,DFW,...,CLR,10SM,44.0,24.0,45.0,4.0,320,0.0,28.89,20041124039501753
7857,MTJ_DEN_35253,2004,9,8,3,640,744,64,MTJ,DEN,...,CLR,10SM,52.0,39.0,61.0,5.0,190,0.0,24.51,20040908930130653
7858,JNU_SEA_32316,2004,3,18,4,1421,1738,137,JNU,SEA,...,FEW050 BKN200,10SM,39.0,11.0,31.0,4.0,100,0.0,29.29,20040318253091453
7859,DRO_DEN_27288,2004,11,17,3,800,914,74,DRO,DEN,...,SCT001 OVC006,0.25SM,,,,0.0,000,0.0,,20041117930050801


In [236]:
origin_dest_df = pd.concat([origin_df,dest_hourly_data], axis=1)

In [237]:
origin_dest_df

,FlightNumber,Year,Month,DayOfMonth,DayOfWeek,ScheduledDepTime,ScheduledArrTime,ScheduledTravelTime,Origin,Destination,...,SkyConditions_d,Visibility_d,DBT_d,DewPointTemp_d,RelativeHumidityPercent_d,WindSpeed_d,WindDirection_d,WindGustValue_d,StationPressure_d,new_d
0,BOS_CLE_29089,2004,9,20,1,1000,1154,114,BOS,CLE,...,FEW250,10SM,71.0,35.0,27.0,8.0,190,0.0,29.49,20040920148201151
1,BOS_OAK_55049,2004,9,10,5,820,1200,400,BOS,OAK,...,CLR,10SM,69.0,54.0,59.0,11.0,290,0.0,29.86,20040910232301153
2,BOS_RSW_47586,2004,3,22,1,700,1036,216,BOS,RSW,...,FEW019,10SM,,,,6.0,050,0.0,,20040322128941030
3,BOS_PIT_49759,2004,5,1,6,845,1031,106,BOS,PIT,...,SCT031 BKN080,10SM,72.0,59.0,64.0,9.0,220,0.0,28.76,20040501948231051
4,BOS_DCA_32791,2004,7,15,4,1300,1430,90,BOS,DCA,...,SCT060 BKN095,10SM,83.0,56.0,40.0,13.0,260,0.0,29.69,20040715137431451
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7856,LAW_DFW_46776,2004,11,24,3,1756,1857,61,LAW,DFW,...,CLR,10SM,49.0,33.0,55.0,9.0,290,0.0,29.45,20041124039271853
7857,MTJ_DEN_35253,2004,9,8,3,640,744,64,MTJ,DEN,...,FEW080 SCT130,10SM,68.0,42.0,39.0,10.0,240,0.0,24.77,20040908030170753
7858,JNU_SEA_32316,2004,3,18,4,1421,1738,137,JNU,SEA,...,FEW040 SCT050 BKN100,10SM,43.0,30.0,60.0,17.0,230,24.0,29.34,20040318242331756
7859,DRO_DEN_27288,2004,11,17,3,800,914,74,DRO,DEN,...,FEW080 SCT120 BKN250,10SM,51.0,30.0,45.0,6.0,250,0.0,24.79,20041117030170953


In [238]:
origin_dest_df.columns

Index(['FlightNumber', 'Year', 'Month', 'DayOfMonth', 'DayOfWeek',
       'ScheduledDepTime', 'ScheduledArrTime', 'ScheduledTravelTime', 'Origin',
       'Destination', 'Distance', 'ActualArrivalTimeStamp', 'result', 'delay',
       'delay_in_seconds', 'target', 'WeatherStationID_origin',
       'AirportID_origin', 'GroundHeight_origin', 'StationHeight_origin',
       'BarometerHeight_origin', 'Latitude_origin', 'Longitude_origin',
       'TimeZone_origin', 'WeatherStationID_dest', 'AirportID_dest',
       'GroundHeight_dest', 'StationHeight_dest', 'BarometerHeight_dest',
       'Latitude_dest', 'Longitude_dest', 'TimeZone_dest',
       'WeatherStationID_o', 'YearMonthDay_o', 'Time_o', 'SkyConditions_o',
       'Visibility_o', 'DBT_o', 'DewPointTemp_o', 'RelativeHumidityPercent_o',
       'WindSpeed_o', 'WindDirection_o', 'WindGustValue_o',
       'StationPressure_o', 'new_o', 'WeatherStationID_d', 'YearMonthDay_d',
       'Time_d', 'SkyConditions_d', 'Visibility_d', 'DBT_d', 'DewPoint

In [239]:
origin_dest_df.to_csv('origin_dest_df.csv')